---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [201]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [87]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    # load data
    towns = pd.read_table('university_towns.txt', names=['State'])
    
    # new column
    towns['RegionName'] = 0
    
    # for every row
    for i in range(len(towns)):
        
        # if it's a state
        if '[edit]' in towns['State'].iloc[i]:
            
            # get and clean current state name
            state = towns['State'].iloc[i].split('[')[0]  
            
            # set 'State' value to current state name
            towns['State'].iloc[i] = state                
            
        # if it's a region name   
        else:
            
            # clean region name  
            towns['RegionName'].iloc[i] = towns['State'].iloc[i].split(' (')[0]
            
            # set 'State' value to current state name
            towns['State'].iloc[i] = state
            
    # remove lines without region name    
    towns = towns[towns['RegionName'] != 0]
    
    return towns


get_list_of_university_towns().head(10)

/opt/conda/lib/python3.6/site-packages/pandas/core/indexing.py:141: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


,State,RegionName
1,Alabama,Auburn
2,Alabama,Florence
3,Alabama,Jacksonville
4,Alabama,Livingston
5,Alabama,Montevallo
6,Alabama,Troy
7,Alabama,Tuscaloosa
8,Alabama,Tuskegee
10,Alaska,Fairbanks
12,Arizona,Flagstaff


In [118]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    
    # load data
    gdp = pd.read_excel('gdplev.xls', skiprows=7, usecols=[4,6], names=['Quarter', 'GDP'])
    
    # drop rows before 2000q1
    gdp = gdp[gdp['Quarter'] >= '2000q1'].set_index('Quarter')
    
    # find recession start
    for i in range(len(gdp)):
        if ((gdp.iloc[i-2]['GDP'] > gdp.iloc[i-1]['GDP']) & (gdp.iloc[i-1]['GDP'] > gdp.iloc[i]['GDP'])):
        
            return gdp.index[i-1]  # (for correct answer it should be 1st quarter of 2 quarters)
        
    

get_recession_start()


'2008q3'

In [120]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    
    # load data
    gdp = pd.read_excel('gdplev.xls', skiprows=7, usecols=[4,6], names=['Quarter', 'GDP'])
    
    # drop rows before recession start
    gdp = gdp[gdp['Quarter'] >= get_recession_start()].set_index('Quarter')
    
    # find recession end
    for i in range(len(gdp)):
        if ((gdp.iloc[i-2]['GDP'] < gdp.iloc[i-1]['GDP']) & (gdp.iloc[i-1]['GDP'] < gdp.iloc[i]['GDP'])):
        
            return gdp.index[i]  # (this time it should be 2nd quarter of 2 quarters)
    

get_recession_end()

'2009q4'

In [126]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    
    # load data
    gdp = pd.read_excel('gdplev.xls', skiprows=7, usecols=[4,6], names=['Quarter', 'GDP'])
    
    # drop rows before recession start and after recession end
    gdp = gdp[(gdp['Quarter'] >= get_recession_start()) &
              (gdp['Quarter'] <= get_recession_end())].set_index('Quarter')
    
    # find recession bottom
    bottom = gdp['GDP'].argmin()
    
    return bottom
                   

get_recession_bottom()

'2009q2'

In [215]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    
    import numpy as np
    
    # get quarters names
    gdp = pd.read_excel('gdplev.xls', skiprows=7, usecols=[4,6], names=['Quarter', 'GDP'])
    quarters = gdp['Quarter'][gdp['Quarter'] >= '2000q1'].tolist()
    quarters.append('2016q3')
    
    # load data
    prices = pd.read_csv('City_Zhvi_AllHomes.csv')
    
    # map state names to two letter acronyms
    prices['State'] = prices['State'].replace(states)
    
    # sort and set in multi-index
    prices = prices.sort_values(['State','RegionName']).set_index(['State','RegionName'])
    
    # dictionary to map month numbers to two quarter numbers
    quarter_to_month = {'1':['-01','-02','-03'], 
                        '2':['-04','-05','-06'], 
                        '3':['-07','-08','-09'], 
                        '4':['-10','-11','-12']}
    
    # count quarters means and save to new columns
    for q in quarters:
        prices[q] = np.mean(pd.concat([prices.get([q[:4] + quarter_to_month[q[-1]][0]]),
                                       prices.get([q[:4] + quarter_to_month[q[-1]][1]]),
                                       prices.get([q[:4] + quarter_to_month[q[-1]][2]])], axis=1), axis=1)
                
    
    # leave only useful columns
    prices = prices[quarters]
    
    return prices


convert_housing_data_to_quarters().head(10)

2000q1         2000q2         2000q3  \
State   RegionName                                                    
Alabama Adamsville       69033.333333   69166.666667   69800.000000   
        Alabaster       122133.333333  123066.666667  123166.666667   
        Albertville      73966.666667   72600.000000   72833.333333   
        Arab             83766.666667   81566.666667   81333.333333   
        Ardmore                   NaN            NaN            NaN   
        Axis                      NaN            NaN            NaN   
        Baileyton                 NaN            NaN            NaN   
        Bay Minette      81700.000000   78533.333333   79133.333333   
        Bayou La Batre   44066.666667   44500.000000   44266.666667   
        Bessemer                  NaN            NaN            NaN   

                               2000q4         2001q1         2001q2  \
State   RegionName                                                    
Alabama Adamsville       71966.666667   73466.666667   74000.000000   
        Alabaster       123700.000000  123233.333333  125133.333333   
        Albertville      74200.000000   75900.000000   76000.000000   
        Arab             82966.666667   84200.000000   84533.333333   
        Ardmore                   NaN            NaN            NaN   
        Axis                      NaN            NaN            NaN   
        Baileyton                 NaN            NaN            NaN   
        Bay Minette      81300.000000   85700.000000   87266.666667   
        Bayou La Batre   43666.666667   42500.000000   43333.333333   
        Bessemer                  NaN            NaN            NaN   

                               2001q3         2001q4         2002q1  \
State   RegionName                                                    
Alabama Adamsville       73333.333333   73100.000000   73333.333333   
        Alabaster       127766.666667  127200.000000  127300.000000   
        Albertville      72066.666667   73566.666667   76533.333333   
        Arab             81666.666667   83900.000000   87266.666667   
        Ardmore                   NaN            NaN            NaN   
        Axis                      NaN            NaN            NaN   
        Baileyton                 NaN            NaN            NaN   
        Bay Minette      85900.000000   85000.000000   84066.666667   
        Bayou La Batre   45433.333333   45400.000000   45400.000000   
        Bessemer                  NaN            NaN            NaN   

                               2002q2    ...            2014q2         2014q3  \
State   RegionName                       ...                                    
Alabama Adamsville       73133.333333    ...      77066.666667   75966.666667   
        Alabaster       128000.000000    ...     147133.333333  147633.333333   
        Albertville      76366.666667    ...      84033.333333   84766.666667   
        Arab             87700.000000    ...     113366.666667  111700.000000   
        Ardmore                   NaN    ...     140533.333333  139566.666667   
        Axis                      NaN    ...     111066.666667  111833.333333   
        Baileyton                 NaN    ...      87666.666667   90033.333333   
        Bay Minette      84566.666667    ...     113333.333333  110600.000000   
        Bayou La Batre   45566.666667    ...      49800.000000   51000.000000   
        Bessemer                  NaN    ...      54600.000000   55333.333333   

                               2014q4         2015q1         2015q2  \
State   RegionName                                                    
Alabama Adamsville       71900.000000   71666.666667   73033.333333   
        Alabaster       148700.000000  148900.000000  149566.666667   
        Albertville      86800.000000   88466.666667   89500.000000   
        Arab            111600.000000  110166.666667  109433.333333   
        Ardmore         140900.000000  143233.333333  143000.000000   
  

In [284]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    from scipy.stats import ttest_ind
    
    # load house prices data, count ratio between recession start and bottom
    prices = convert_housing_data_to_quarters()
    prices['Ratio'] = prices[get_recession_bottom()] / prices[get_recession_start()]
    
    # load university towns data
    university_towns = get_list_of_university_towns().set_index(['State', 'RegionName'])
    university_towns['University town'] = True
    
    # merge data, replace NaN's
    all_towns = pd.merge(prices[['Ratio']], university_towns, left_index=True, right_index=True, how='outer')
    all_towns['University town'] = all_towns['University town'].replace(np.NaN, False)
    all_towns = all_towns.dropna(axis=0, how='any')
    
    # separate data and count Ratio means
    uni_towns = all_towns['Ratio'][all_towns['University town'] == True]
    non_uni_towns = all_towns['Ratio'][all_towns['University town'] == False]
    uni_towns_mean = np.mean(uni_towns)
    non_uni_towns_mean = np.mean(non_uni_towns)
    
    # ttesting
    statistic, p = ttest_ind(uni_towns, non_uni_towns)
    
    # get answers
    different = p < 0.01
    better = ['university town', 'non-university town'][np.argmax([uni_towns_mean, non_uni_towns_mean])]
    
    return different, p, better


run_ttest()

/opt/conda/lib/python3.6/site-packages/pandas/core/indexing.py:141: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


(True, 0.0089755875407122598, 'university town')